In [8]:
def generate_verilog(num_inputs):
    weights = ', '.join(str(i) for i in range(1, num_inputs+2))  # +1 for bias

    # Generate the multiplier instances
    multipliers = []
    for i in range(num_inputs+1):
        multipliers.append(f"""
            multiplier #(
                .weigths(weigths[{i}])
            ) multiplier_inst (
                .clk(clk),
                .data_in(input_in[{i}]),
                .data_out(accumulate[{i}])
            );
        """)

    # Generate the sum accumulations
    accumulations = []
    for i in range(num_inputs+1):
        accumulations.append(f"accumulate[{i}]")

    accumulations_str = ' + '.join(accumulations)

    verilog_code = f"""
module neuron_input{num_inputs} #(
    parameter integer weigths [{num_inputs}:0] = '{{
        {weights}
    }}' // {num_inputs} weigth + 1 bias
) (
    input logic clk,
    input integer input_in [{num_inputs-1}:0],
    output integer output_out
);

    logic [15:0] sumAdress;
    logic [7:0] afterActivation;
    integer sumForActivation = 0;

    integer accumulate [{num_inputs-1}:0];

    integer sum;

    genvar i;
    generate
        for (i = 0; i<{num_inputs}; i++) begin : gen_multipliers
            multiplier #(
                .weigths(weigths[i])
            ) multiplier_inst (
                .clk(clk),
                .data_in(input_in[i]),
                .data_out(accumulate[i])
            );
        end
    endgenerate
    
    always_ff @( posedge clk ) begin : acummulations
        sum <= {accumulations_str};
        sumForActivation <= sum + weigths[{num_inputs}];
    end

    always_ff @( posedge clk ) begin : activation
        if (sumForActivation < -32768) begin
            sumAdress <= 0;
        end else if(sumForActivation > 32767) begin
            sumAdress <= ~0;
        end else begin
            sumAdress <= sumForActivation;
        end
    end

    //FOR MODELSIM SIMULATION & QUARTUS SYNTHESIS
   sigmoid_IP sigmoid_inst (
       .clock(clk),
       .address(sumAdress[15:4]), //12 bits
       .q(afterActivation)
   );

   //FOR VIVADO SYNTHESIS
    // sigmoid_IP sigmoid_inst (
    //         .clk(clk),
    //         .a(sumAdress[15:4]), //12 bits
    //         .spo(afterActivation)
    //     );

    assign output_out = afterActivation;

endmodule
    """
    return verilog_code

num_inputs = 64  # Change this to generate code for a different number of inputs
verilog_code = generate_verilog(num_inputs)
# Save the Verilog code to a file
with open(f"neuron_input{num_inputs}.sv", "w") as f:
    f.write(verilog_code)